In [2]:
'''


'''
def create_data(file_path: str):

    data = {}

    with open(file_path, 'r') as f:
        if f == None:
            print('File not found')
            return None
                
        data["num_subjects"] = int(f.readline())
        data["num_students_per_subject"] = [int(x) for x in f.readline().split()]
        data["num_rooms"] = int(f.readline())
        data["num_seats_per_room"] = [int(x) for x in f.readline().split()]
        data["num_pairs"] = int(f.readline())
        data["conflicts"] = []
        for i in range (data["num_pairs"]):
            s1, s2 = [int(x) for x in f.readline().split()]
            data["conflicts"].append((s1, s2))
        
    return data

In [5]:

from ortools.sat.python import cp_model
import math

def cp(num_subjects: int, num_rooms: int, nums_student_per_subject: int, 
        num_seats_per_room: int, subject_pairs: int,
        num_sections_per_day: int = None, num_days: int = None, 
        save_model: bool = False):
    
    if (not num_sections_per_day):
        num_sections_per_day = 4

    if (not num_days):
        num_days = math.ceil(num_subjects / num_sections_per_day)

    model = cp_model.CpModel()

    # Variables x[i] represent the section assigned to subject i
    x = [model.NewIntVar(0, num_subjects, "x[%i]" % i) for i in range(num_subjects)]

    # Variables y[i][j] represent whether subject i is assigned to room j
    y = [[model.NewIntVar(0, 1, "y[%i][%i]" % (i, j)) for j in range(num_rooms)] for i in range(num_subjects)]

    # Objective variable
    obj_var = model.NewIntVar(0, num_subjects, "num_sections")

    # Constraint 1: Each subject must be assigned once
    for i in range(num_subjects):
        model.Add(sum([y[i][j] for j in range(num_rooms)]) == 1)

    # Constraint 2: Each room must have most one subject in each section 
    # (i.e. two subjects cannot be assigned to the same room in the same section)
    # it means x[i] == x[j] => y[i][k] + y[j][k] <= 1 for all k in range(num_rooms)
    for i in range(num_subjects):
        for j in range(num_subjects):
            if i == j:
                continue
            for k in range(num_rooms):
                b = model.NewBoolVar("b[%i][%i]" % (i, j))
                model.Add(x[i] == x[j]).OnlyEnforceIf(b)
                model.Add(x[i] != x[j]).OnlyEnforceIf(b.Not())                                                  
                model.Add(y[i][k] + y[j][k] <= 1).OnlyEnforceIf(b)


    # Constraint 3: Each subject assigned to a room must have enough seats
    for i in range(num_subjects):
        model.Add(sum([y[i][j] * num_seats_per_room[j] for j in range(num_rooms)]) >= nums_student_per_subject[i])

    # Constraints 4: Two subjects cannot be assigned to the same room in the same section
    for i1, i2 in subject_pairs:
        model.Add(x[i1] != x[i2])

    # Constraint 5: Objective variable object_var = max(x[i] all i in range(num_subjects)))
    for i in range(num_subjects):
        model.Add(obj_var >= x[i])

    # Objective: Minimize the number of sections
    model.Minimize(obj_var)

    
    if save_model:
        with open('cp.txt', 'w') as f:
            model.ExportToFile(f.name)
    

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        solution = []
        
        print("Optimal solution found")
        print("Number of sections: ", solver.Value(obj_var))
 
        for subject in range(num_subjects):
            
            room = None
            
            for j in range(num_rooms):
                if solver.Value(y[subject][j]) == 1:
                    room = j
                    break
            
            section_id = solver.Value(x[subject])
            
            print('Subject %i is assigned to room %i in section %i' % (subject, room, section_id))
            
            
            solution.append((subject, room, section_id // num_sections_per_day, section_id % num_sections_per_day))
        
        solution.sort(key=lambda x: x[2] * num_sections_per_day + x[3])
        
        solution_str = ''        
        solution_str += "subjects,number_student,rooms,num_seat,day,section" + "\n"
        for subjects, room, day, section in solution:
            solution_str += str(subjects) + "," + str(nums_student_per_subject[subjects]) + "," + str(room) + "," + \
                str(num_seats_per_room[room]) + "," + str(day) + "," + str(section) + "\n"

        return solution_str

    else:
        print('The problem does not have an optimal solution.')          
        
        

In [6]:
import os

path_to_data = '../data/'

file_name = 'data_10_2_(0).txt'

list_file_name = []

data = create_data(path_to_data + file_name)

solution_str = cp(num_subjects=data["num_subjects"], num_rooms=data["num_rooms"], 
                        nums_student_per_subject=data["num_students_per_subject"],
                        num_seats_per_room=data["num_seats_per_room"], subject_pairs=data["conflicts"])

output_path = "solution/" + file_name[:-4] + "_solution.csv"

with open(output_path, 'w') as f:
    f.write(solution_str)

Optimal solution found
Number of sections:  5
Subject 0 is assigned to room 1 in section 4
Subject 1 is assigned to room 1 in section 0
Subject 2 is assigned to room 1 in section 3
Subject 3 is assigned to room 1 in section 5
Subject 4 is assigned to room 0 in section 4
Subject 5 is assigned to room 0 in section 2
Subject 6 is assigned to room 0 in section 3
Subject 7 is assigned to room 1 in section 1
Subject 8 is assigned to room 1 in section 2
Subject 9 is assigned to room 0 in section 0
